# Navigation Viewer

---

In this notebook, you will be able to run the trained agent

In [ ]:
import os
import sys

p1_navigation_folder_path = os.path.abspath(os.path.join('../'))
sys.path.append(p1_navigation_folder_path)

# add unityagents, 
# this is a copy of the 'python' folder from udacity eep-reinforcement-learning nanodegree
# https://github.com/udacity/deep-reinforcement-learning#dependencies
unityagents_module_path = os.path.abspath(os.path.join('udacity-ml-agents'))
sys.path.append(unityagents_module_path)

# Import necessary packages

In [ ]:
import numpy as np
from collections import deque
import torch

import matplotlib.pyplot as plt

from unityagents import UnityEnvironment
from dqn_agent import Agent

Use the same file name as in the 'Navigation' notebook

In [ ]:
env = UnityEnvironment(file_name="../Banana_Windows_x86_64/Banana.x86_64")

# Reset and examine the Environment

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Execute Trained Agent in the Environment



In [ ]:
agent = Agent(state_size=state_size, action_size=action_size, seed=0)
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('model.pth'))

# run only one episode
def evaluate_trained_model(n_episodes=1, max_t=1000, eps_start=0.2, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []  # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start  # initialize epsilon
    for i_episode in range(1, n_episodes + 1):

        env_info = env.reset(train_mode=False)[brain_name]  # reset the environment
        state = env_info.vector_observations[0]  # get the current state
        score = 0  # initialize the score

        for t in range(max_t):
            action = agent.act(state, eps)  # select an action
            env_info = env.step(action.astype(int))[brain_name]  # send the action to the environment
            # time.sleep(0.05)
            next_state = env_info.vector_observations[0]  # get the next state
            reward = env_info.rewards[0]  # get the reward
            done = env_info.local_done[0]  # see if episode has finished

            score += reward  # update the score
            state = next_state  # roll over the state to next time step
            if done:  # exit loop if episode finished
                break
        scores_window.append(score)  # save most recent score
        scores.append(score)  # save most recent score
        eps = max(eps_end, eps_decay * eps)  # decrease epsilon

        print('\rEpisode {}\t Score: {:.2f}'.format(i_episode, score))

    env.close()

    return scores

scores = evaluate_trained_model()